In [1]:
from os import sys
# Path to workspace
sys.path.insert(0, '/workspace/3d-shapes-embeddings/contrib/sharp_features/')
sys.path.insert(0, '/workspace/dense-self-supervised-representation-learning-for-3D-shapes/')

import h5py
import torch
import numpy as np
from tqdm import tqdm
import k3d

In [2]:
import neptune.new as neptune
from workspace.utils.train_loop import *

params = {
    'name': 'Crossmodal pretrained',
    'dataset': 'abc',
    'batch_size': 5,
    'tau': 0.07,
    'n_output': 512,
    'result_dim': 128,
    'hidden_dim': 256,
    'total_epochs': 50,
    'lr': 5e-5,
    'weight_decay': 1e-5,
    'save_every': 50,
    'weights_root': '../weights/'
}

# tags
tags = ['abc']



logger = neptune.init(project='seals5454/crossmodal-exps-igor',
                      name=params['name'],
                      tags=tags,
                      api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmU'\
                                'uYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS'\
                                '5haSIsImFwaV9rZXkiOiI2NTIwODVkNC1hOTg5LTQ4NTAtY'\
                                'WRhNS0yMGY4MmQ1YzBmZWIifQ=='
                      )

logger['parameters'] = params

device1, device2, device = 'cuda:3', 'cuda:3', 'cuda:3'

/opt/conda/lib/python3.8/site-packages/neptune/new/internal/utils/git.py:35: UserWarning: GitPython could not be initialized
  warnings.warn("GitPython could not be initialized")
/opt/conda/lib/python3.8/site-packages/neptune/new/internal/utils/git.py:35: UserWarning: GitPython could not be initialized
  warnings.warn("GitPython could not be initialized")


https://app.neptune.ai/seals5454/crossmodal-exps-igor/e/IGOREXP-47
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [3]:
from torch.utils.data import DataLoader
from workspace.crossmodal.data.datasets import *
from workspace.datasets.transforms import *
from workspace.crossmodal.utils.collates import collate_clouds, collate_meshes, multicollate

collate = lambda data: multicollate(
    data,
    lambda x: collate_meshes(x, device=device),
    lambda x: collate_meshes(x, device=device),
    lambda x: collate_clouds(x, device=device),
    lambda x: collate_clouds(x, device=device),
)

pdataset_train = DoubleDataset(data_path='abc_train.hdf5', modality=Modality.POINT_CLOUD,
                            transform=Compose(
        PointCloudNormalize(),
        RandomRotation(low=-45, high=45, axis='xyz'),
        RandomJitter(std=0.01, clip_bound=0.05)
    ),)
mdataset_train = DoubleDataset(data_path='abc_train.hdf5', modality=Modality.MESH,
                            transform=Compose(
        #MeshNetRandomRotation(low=-45, high=45, axis='xyz'),
        MeshNetRandomJitter(std=0.01, clip_bound=0.05)
    ),)

train = DoubleModalityDataset(mdataset_train, pdataset_train)


pdataset_test = DoubleDataset(data_path='abc_test.hdf5', modality=Modality.POINT_CLOUD,
                            transform=Compose(
        PointCloudNormalize(),
        RandomRotation(low=-45, high=45, axis='xyz'),
        RandomJitter(std=0.01, clip_bound=0.05)
    ),)
mdataset_test = DoubleDataset(data_path='abc_test.hdf5', modality=Modality.MESH,
                             transform=Compose(
        #MeshNetRandomRotation(low=-45, high=45, axis='xyz'),
        MeshNetRandomJitter(std=0.01, clip_bound=0.05)
    ),)

test = DoubleModalityDataset(mdataset_test, pdataset_test)



train_loader = DataLoader(train, batch_size=params['batch_size'], shuffle=True,
               collate_fn=collate
)

test_loader = DataLoader(test, batch_size=params['batch_size'], shuffle=False,
              collate_fn=collate
)

In [4]:
class Transpose(torch.nn.Module):
    def __init__(self, *dims):
        super().__init__()
        self.dims = dims

    def forward(self, data):
        return data.transpose(*self.dims)
    

class MultiModalModel(torch.nn.Module):
    def __init__(self, model1, model2, model_output_dim, result_dim=128, hidden_dim=256):
        super().__init__()
        self.model1 = model1.to(device1)
        self.model2 = model2.to(device2)
        self.head1 = torch.nn.Sequential(
            Transpose(1, 2),
            torch.nn.Linear(model_output_dim, hidden_dim),
            Transpose(1, 2),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            Transpose(1, 2),
            torch.nn.Linear(hidden_dim, result_dim),
            Transpose(1, 2),
        ).to(device1)
        
        self.head2 = torch.nn.Sequential(
            Transpose(1, 2),
            torch.nn.Linear(model_output_dim, hidden_dim),
            Transpose(1, 2),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            Transpose(1, 2),
            torch.nn.Linear(hidden_dim, result_dim),
            Transpose(1, 2),
        ).to(device2)
        
        
    def forward(self, input1_1, input1_2, input2_1, input2_2):
        v1_1_emb = self.model1.forward_features(input1_1)
        v1_2_emb = self.model1.forward_features(input1_2)
        v2_1_emb = self.model2.forward_features(input2_1)
        v2_2_emb = self.model2.forward_features(input2_2)
        
        
        return (
            self.head1(v1_1_emb),
            self.head1(v1_2_emb),
            self.head2(v2_1_emb),
            self.head2(v2_2_emb),
        )
    

    def get_embeddings(self, input1_1, input1_2, input2_1, input2_2):
        v1_1_emb = self.model1.forward_features(input1_1)
        v1_2_emb = self.model1.forward_features(input1_2)
        v2_1_emb = self.model2.forward_features(input2_1)
        v2_2_emb = self.model2.forward_features(input2_2)
        
        return v1_1_emb, v1_2_emb, v2_1_emb, v2_2_emb
    
    
class OneModalityModel(torch.nn.Module):
    def __init__(self, model, model_output_dim, result_dim=128, hidden_dim=256):
        super().__init__()
        self.model = model.to(device1)
        self.head = torch.nn.Sequential(
            Transpose(1, 2),
            torch.nn.Linear(model_output_dim, hidden_dim),
            Transpose(1, 2),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            Transpose(1, 2),
            torch.nn.Linear(hidden_dim, result_dim),
            Transpose(1, 2),
        ).to(device1)
        
    def forward(self, input1, input2):
        v1_emb = self.model.forward_features(input1)
        v2_emb = self.model.forward_features(input2)
        
        
        return (
            self.head(v1_emb),
            self.head(v2_emb),
        )
    

    def get_embeddings(self, input1_1, input1_2, input2_1, input2_2):
        v1_emb = self.model.forward_features(input1)
        v2_emb = self.model.forward_features(input2)
        
        return v1_emb, v2_emb

In [5]:
checkpoints_path = '../weights/'

In [6]:
mnet_sd = torch.load(checkpoints_path + 'IGOREXP-45/50epoch.pt')
dgcnn_sd = torch.load(checkpoints_path + 'IGOREXP-44/50epoch.pt')

In [7]:
from workspace.models.meshnet import MeshNet
from workspace.models.dgcnn import DGCNN

# mnet = MeshNet(n_patches=5)
# dgcnn = DGCNN(n_patches=5)

mnet = OneModalityModel(
    MeshNet(n_patches=5),
    params['n_output'],
    result_dim=params['result_dim'],
    hidden_dim=params['hidden_dim']
)

dgcnn = OneModalityModel(
    DGCNN(n_patches=5),
    params['n_output'],
    result_dim=params['result_dim'],
    hidden_dim=params['hidden_dim']
)

mnet.load_state_dict(mnet_sd)
dgcnn.load_state_dict(dgcnn_sd)

model = MultiModalModel(
    mnet.model,
    dgcnn.model,
    params['n_output'],
    result_dim=params['result_dim'],
    hidden_dim=params['hidden_dim']
)
#model = MultiModalModel(mnet, dgcnn, 512)

In [8]:
#from copy import deepcopy
from workspace.crossmodal.utils.losses import *

def forward( 
    model,
    batch, # raw data from dataloader
    logger, # neptune run
    mode # 'train'/'val'
): # -> loss

    data1_1, data1_2, (data2_1, face_indexes), (data2_2, _) = batch
    
    max_faces = data1_1[0].shape[-1]

    out1_1, out1_2, out2_1, out2_2 = model(data1_1, data1_2, data2_1, data2_2)
    
    pooled2_1, counts2_1 = get_patch_embeddings(out2_1, face_indexes, max_faces)
    pooled2_2, counts2_2 = get_patch_embeddings(out2_2, face_indexes, max_faces)
    face_counts = face_indexes_to_patch_counts(face_indexes, max_faces)
    
    #local inside figures
    pc_local_loss = patch_contrastive_loss(
        (pooled2_1, counts2_1),
        (pooled2_2, counts2_2),
        params
    )
    mesh_local_loss = patch_contrastive_loss(
        (out1_1, face_counts),
        (out1_2, face_counts),
        params
    )
    
    #local crossmodal loss
    local_crossmodal_loss = (
        patch_contrastive_loss(
            (pooled2_1, counts2_1),
            (out1_1, face_counts),
            params
        ) + 
        patch_contrastive_loss(
            (pooled2_1, counts2_1),
            (out1_2, face_counts),
            params
        ) +
        patch_contrastive_loss(
            (pooled2_2, counts2_2),
            (out1_1, face_counts),
            params
        ) + 
        patch_contrastive_loss(
            (pooled2_2, counts2_2),
            (out1_2, face_counts),
            params
        )
    )
    
    
    gout1_1 = out1_1.mean(-1)
    gout1_2 = out1_2.mean(-1)
    gout2_1 = out2_1.mean(-1)
    gout2_2 = out2_2.mean(-1)
    # crossmodal
    crossmodal_loss = contrastive_loss(gout1_1, gout2_1, params) +\
           contrastive_loss(gout1_2, gout2_2, params) +\
           contrastive_loss(gout1_1, gout2_2, params) +\
           contrastive_loss(gout1_2, gout2_1, params)
    
    # model level
    pc_loss = contrastive_loss(gout1_1, gout1_2, params)
    mesh_loss = contrastive_loss(gout2_1, gout2_2, params)
    
    
    return {
        'loss': (0.25 * crossmodal_loss + pc_loss + mesh_loss) +\
                (0.25 * local_crossmodal_loss + pc_local_loss + mesh_local_loss),
        'pc_loss': pc_loss,
        'pc_local_loss': pc_local_loss,
        'mesh_local_loss': mesh_local_loss,
        'mesh_loss': mesh_loss,
        'local_crossmodal_loss': 0.25 * local_crossmodal_loss,
        'crossmodal_loss:': 0.25 * crossmodal_loss
    }

In [9]:
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=params['lr'],
    weight_decay=params['weight_decay']
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_loader) * params['total_epochs'])

In [10]:
train_model(model, params, logger,  train_loader, test_loader, optimizer, scheduler, forward)

Validation: 100%|██████████| 40/40 [00:27<00:00,  1.43it/s, Loss=25.1]


In [ ]:
batch = next(iter(train_loader))
with torch.no_grad():
    activations = model(batch[0], batch[1])
    fm1, fm2 = activations

In [ ]:
fm1 = F.normalize(fm1, dim=-1)
fm2 = F.normalize(fm2, dim=-1)

res = fm1.cpu().mean(-1) @ fm2.cpu().mean(-1).T
`
print(res)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(res, cmap='hot', interpolation='nearest')